In [1]:
import imaplib
import email
import os
import json
import logging
from datetime import datetime, timedelta
from email.header import decode_header
import sys
from pathlib import Path

# Add src to path for imports
sys.path.append('../src')

print("✅ All libraries imported successfully!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ All libraries imported successfully!
📅 Current date: 2025-08-12 12:37:58


In [ ]:
# Configuration for email fetcher
CONFIG = {
    "email_address": "surendharal65@gmail.com",  # Enter your email address
    "password": "ggvgpgpmykcgbphj",       # Enter your app password
    "imap_server": "imap.gmail.com",  # Change based on your provider
    "imap_port": 993,
    "download_folder": "../data/raw_emails",
    "search_criteria": {
        "hdfc_sender": "Emailstatements.cards@hdfcbank.net",
        "idfc_sender": "statement@idfcfirstbank.com",
        "subject_keywords": ["statement", "credit card", "bill", "HDFC", "IDFC", "Your HDFC Bank Titanium Times Card Credit Card Statement",
                             "Your HDFC Bank Insta Jumbo Loan Statement for the month"]
    }
}

# Create necessary directories
os.makedirs(CONFIG['download_folder'], exist_ok=True)
os.makedirs('../config', exist_ok=True)

print("📁 Directory structure created!")
#print("⚠️  IMPORTANT: Please update CONFIG with your email credentials above!")

📁 Directory structure created!


In [5]:
# Setup comprehensive logging
def setup_logging():
    """Setup logging for email fetcher"""
    log_format = '%(asctime)s - %(levelname)s - %(message)s'
    
    # Create logs directory
    os.makedirs('../logs', exist_ok=True)
    
    # Setup file and console logging
    logging.basicConfig(
        level=logging.INFO,
        format=log_format,
        handlers=[
            logging.FileHandler('../logs/email_fetcher.log'),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

# Initialize logger
logger = setup_logging()
logger.info("Email Fetcher Phase 1 Started")

2025-08-12 13:10:46,027 - INFO - Email Fetcher Phase 1 Started


In [7]:
def connect_to_email(config):
    """Connect to email server using IMAP"""
    try:
        # Connect to IMAP server
        mail = imaplib.IMAP4_SSL(config['imap_server'], config['imap_port'])
        
        # Login with credentials
        mail.login(config['email_address'], config['password'])
        
        logger.info(" Successfully connected to email server")
        return mail
        
    except Exception as e:
        logger.error(f" Failed to connect to email: {str(e)}")
        return None

def test_connection():
    """Test email connection with current config"""
    if not CONFIG['email_address'] or not CONFIG['password']:
        print(" Please update CONFIG with your email credentials first!")
        return False
    
    mail = connect_to_email(CONFIG)
    if mail:
        try:
            # Select INBOX before closing
            mail.select("INBOX")
            mail.close()
            mail.logout()
            print(" Email connection test successful!")
            return True
        except:
            pass
    
    print(" Email connection test failed!")
    return False

# Test the connection
test_connection()

2025-08-12 13:17:44,070 - INFO -  Successfully connected to email server


 Email connection test successful!


True

In [18]:
def search_credit_card_emails(mail, config, days_back=365):
    """Search for credit card statement emails"""
    try:
        # Select inbox
        mail.select('inbox')
        
        # Calculate date range (going backwards from today)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back)
        start_date_str = start_date.strftime("%d-%b-%Y")
        
        logger.info(f" Searching emails from {start_date_str} to today")
        
        # Multiple search criteria for better coverage
        search_criteria = [
            f'(FROM "{config["search_criteria"]["hdfc_sender"]}" SINCE "{start_date_str}")',
            f'(FROM "{config["search_criteria"]["idfc_sender"]}" SINCE "{start_date_str}")',
            '(SUBJECT "HDFC Credit Card Statement" SINCE "{}")'.format(start_date_str),
            '(SUBJECT "IDFC Credit Card Statement" SINCE "{}")'.format(start_date_str),
            '(SUBJECT "statement" SINCE "{}") (FROM "hdfc")'.format(start_date_str),
            '(SUBJECT "statement" SINCE "{}") (FROM "idfc")'.format(start_date_str),
        ]
        
        all_email_ids = set()  # Use set to avoid duplicates
        
        for criteria in search_criteria:
            try:
                typ, messages = mail.search(None, criteria)
                if messages[0]:
                    email_ids = messages[0].split()
                    all_email_ids.update(email_ids)
                    logger.info(f"Found {len(email_ids)} emails with criteria: {criteria}")
            except Exception as e:
                logger.warning(f"Search failed for criteria {criteria}: {str(e)}")
        
        unique_email_ids = list(all_email_ids)
        logger.info(f" Total unique emails found: {len(unique_email_ids)}")
        
        return unique_email_ids
        
    except Exception as e:
        logger.error(f" Search failed: {str(e)}")
        return []

# Test search function (will run in next cell with actual connection)
print(" Search functions defined successfully!")

 Search functions defined successfully!


In [19]:
def decode_header_value(header_value):
    """Decode email header value"""
    if header_value:
        try:
            decoded = decode_header(header_value)[0]
            if isinstance(decoded[0], bytes):
                return decoded[0].decode(decoded[1] or 'utf-8')
            return decoded[0]
        except:
            return str(header_value)
    return ""

def identify_bank(sender):
    """Identify bank from sender email"""
    sender_lower = sender.lower()
    if 'hdfc' in sender_lower:
        return 'HDFC'
    elif 'idfc' in sender_lower:
        return 'IDFC'
    elif 'axis' in sender_lower:
        return 'AXIS'
    elif 'sbi' in sender_lower:
        return 'SBI'
    elif 'icici' in sender_lower:
        return 'ICICI'
    else:
        return 'UNKNOWN'

def make_safe_filename(filename):
    """Make filename safe for filesystem"""
    unsafe_chars = '<>:"/\\|?*'
    for char in unsafe_chars:
        filename = filename.replace(char, '_')
    return filename

def is_pdf_attachment(part):
    """Check if email part is a PDF attachment"""
    content_disposition = part.get("Content-Disposition", "")
    content_type = part.get_content_type()
    filename = part.get_filename()
    
    is_attachment = "attachment" in content_disposition
    is_pdf = (content_type == "application/pdf" or 
              (filename and filename.lower().endswith('.pdf')))
    
    return is_attachment and is_pdf

print(" PDF processing functions defined successfully!")

 PDF processing functions defined successfully!


In [10]:
def extract_pdf_attachments(mail, email_ids, config):
    """Extract PDF attachments from emails"""
    pdf_files = []
    total_emails = len(email_ids)
    
    logger.info(f" Processing {total_emails} emails for PDF attachments...")
    
    for idx, email_id in enumerate(email_ids, 1):
        try:
            logger.info(f"Processing email {idx}/{total_emails} (ID: {email_id.decode()})")
            
            # Fetch email
            typ, msg_data = mail.fetch(email_id, '(RFC822)')
            
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    # Parse email message
                    msg = email.message_from_bytes(response_part[1])
                    
                    # Get email details
                    subject = decode_header_value(msg["Subject"])
                    sender = decode_header_value(msg.get("From"))
                    date = msg.get("Date")
                    
                    logger.info(f" Processing: {subject[:50]}... from {sender}")
                    
                    # Process attachments
                    if msg.is_multipart():
                        for part in msg.walk():
                            if is_pdf_attachment(part):
                                pdf_info = save_pdf_attachment(
                                    part, subject, sender, date, config
                                )
                                if pdf_info:
                                    pdf_files.append(pdf_info)
                                    logger.info(f" Downloaded: {pdf_info['filename']}")
                                    
        except Exception as e:
            logger.error(f" Error processing email {email_id}: {str(e)}")
            continue
    
    return pdf_files

def save_pdf_attachment(part, subject, sender, date, config):
    """Save PDF attachment to disk"""
    try:
        filename = part.get_filename()
        if not filename:
            return None
            
        # Clean filename
        safe_filename = make_safe_filename(filename)
        
        # Create timestamp for unique naming
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Determine bank type from sender
        bank_type = identify_bank(sender)
        
        # Create final filename
        final_filename = f"{bank_type}_{timestamp}_{safe_filename}"
        
        # Ensure download directory exists
        download_path = config['download_folder']
        os.makedirs(download_path, exist_ok=True)
        
        # Full file path
        file_path = os.path.join(download_path, final_filename)
        
        # Save file
        with open(file_path, 'wb') as f:
            f.write(part.get_payload(decode=True))
        
        file_size = os.path.getsize(file_path)
        logger.info(f" Saved PDF: {final_filename} ({file_size} bytes)")
        
        return {
            'filename': final_filename,
            'file_path': file_path,
            'bank': bank_type,
            'subject': subject,
            'sender': sender,
            'date': date,
            'size': file_size,
            'download_timestamp': datetime.now().isoformat()
        }
        
    except Exception as e:
        logger.error(f" Error saving PDF: {str(e)}")
        return None

print(" PDF download functions defined successfully!")

 PDF download functions defined successfully!


In [ ]:
def run_complete_email_fetch(config, since_date, before_date):
    """Fetch credit card statement emails between two given dates."""
    logger.info(f" Fetching Credit Card Emails from {since_date} to {before_date}")

    if not config['email_address'] or not config['password']:
        logger.error(" Email credentials not provided in config!")
        return []

    all_pdfs = []
    mail = connect_to_email(config)
    if not mail:
        return []

    try:
        # Convert string dates to datetime objects
        start_date = datetime.strptime(since_date, "%d-%b-%Y")
        end_date = datetime.strptime(before_date, "%d-%b-%Y") - timedelta(days=1)

        # Search emails for this date range
        email_ids = search_credit_card_emails_between(mail, config, start_date, end_date)

        if not email_ids:
            logger.info(f" No emails found in this date range.")
            return []

        # Extract PDFs
        pdf_files = extract_pdf_attachments(mail, email_ids, config)
        all_pdfs.extend(pdf_files)

        # Save summary
        summary = {
            'total_emails_found': len(email_ids),
            'total_pdfs_downloaded': len(all_pdfs),
            'download_timestamp': datetime.now().isoformat(),
            'date_range': {
                'since': since_date,
                'before': before_date
            },
            'pdf_files': all_pdfs
        }
        summary_path = os.path.join(config['download_folder'], f'download_summary.json')
        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=2)

        logger.info(f" Done: {len(email_ids)} emails, {len(pdf_files)} PDFs.")
        return all_pdfs

    finally:
        try:
            mail.close()
            mail.logout()
            logger.info(" Email connection closed")
        except:
            pass


In [20]:
def search_credit_card_emails_between(mail, config, start_date, end_date):
    """Search for credit card statement emails between two dates."""
    try:
        mail.select('inbox')
        start_str = start_date.strftime("%d-%b-%Y")
        end_str = end_date.strftime("%d-%b-%Y")

        logger.info(f"Searching emails from {start_str} to {end_str}")

        search_criteria = [
            f'(FROM "{config["search_criteria"]["hdfc_sender"]}" SINCE "{start_str}" BEFORE "{end_str}")',
            f'(FROM "{config["search_criteria"]["idfc_sender"]}" SINCE "{start_str}" BEFORE "{end_str}")',
            f'(SUBJECT "HDFC Credit Card Statement" SINCE "{start_str}" BEFORE "{end_str}")',
            f'(SUBJECT "IDFC Credit Card Statement" SINCE "{start_str}" BEFORE "{end_str}")',
            f'(SUBJECT "statement" SINCE "{start_str}" BEFORE "{end_str}") (FROM "hdfc")',
            f'(SUBJECT "statement" SINCE "{start_str}" BEFORE "{end_str}") (FROM "idfc")',
        ]

        all_email_ids = set()

        for criteria in search_criteria:
            try:
                typ, messages = mail.search(None, criteria)
                if messages[0]:
                    email_ids = messages[0].split()
                    all_email_ids.update(email_ids)
                    logger.info(f"Found {len(email_ids)} emails with criteria: {criteria}")
            except Exception as e:
                logger.warning(f"Search failed for criteria {criteria}: {str(e)}")

        return list(all_email_ids)

    except Exception as e:
        logger.error(f"Search failed: {str(e)}")
        return []


In [21]:
def run_yearly_email_fetch(CONFIG, start_year=None):
    """Fetch credit card statement emails year-by-year until no results are found."""
    if not CONFIG['email_address'] or not CONFIG['password']:
        print("STOP! Please update CONFIG with your email credentials first!")
        return

    if start_year is None:
        start_year = datetime.now().year  # Current year

    current_year = start_year
    all_downloaded_files = []

    print("Starting Year-by-Year Email Fetching Process...")
    print(f"Email: {CONFIG['email_address']}")
    print("=" * 50)

    while True:
        since_date = f"01-Jan-{current_year}"
        before_date = f"01-Jan-{current_year + 1}"

        print(f"\n Processing Year: {current_year}")
        print(f"Searching from {since_date} to {before_date}...")

        downloaded_files = run_complete_email_fetch(
            CONFIG,
            since_date=since_date,
            before_date=before_date
        )

        if not downloaded_files:
            print(f" No emails found for {current_year}. Stopping search.")
            break

        print(f" {len(downloaded_files)} PDFs downloaded for {current_year}")
        all_downloaded_files.extend(downloaded_files)

        import time
        time.sleep(3)  # Avoid Gmail rate limits

        current_year -= 1

    print("=" * 50)
    print(" All Years Processing Complete!")
    print(f"Total PDFs downloaded: {len(all_downloaded_files)}")
    return all_downloaded_files


# Run it
all_files = run_yearly_email_fetch(CONFIG)


2025-08-12 18:19:07,726 - INFO -  Fetching Credit Card Emails from 01-Jan-2025 to 01-Jan-2026


Starting Year-by-Year Email Fetching Process...
Email: surendharal65@gmail.com

 Processing Year: 2025
Searching from 01-Jan-2025 to 01-Jan-2026...


2025-08-12 18:19:09,229 - INFO -  Successfully connected to email server
2025-08-12 18:19:09,740 - INFO - Searching emails from 01-Jan-2025 to 31-Dec-2025
2025-08-12 18:19:10,344 - INFO - Found 5 emails with criteria: (FROM "Emailstatements.cards@hdfcbank.net" SINCE "01-Jan-2025" BEFORE "31-Dec-2025")
2025-08-12 18:19:11,294 - INFO - Found 5 emails with criteria: (SUBJECT "statement" SINCE "01-Jan-2025" BEFORE "31-Dec-2025") (FROM "hdfc")
2025-08-12 18:19:11,598 - INFO - Found 17 emails with criteria: (SUBJECT "statement" SINCE "01-Jan-2025" BEFORE "31-Dec-2025") (FROM "idfc")
2025-08-12 18:19:11,601 - INFO -  Processing 22 emails for PDF attachments...
2025-08-12 18:19:11,604 - INFO - Processing email 1/22 (ID: 84442)
2025-08-12 18:19:12,664 - INFO -  Processing: Your FIRST Millennia Credit Card Statement... from IDFC FIRST Bank <statement@idfcfirstbank.com>
2025-08-12 18:19:12,687 - INFO -  Saved PDF: IDFC_20250812_181912_20000002170120_22072025_111137192.pdf (512464 bytes)
2025-08-1

 22 PDFs downloaded for 2025


2025-08-12 18:19:34,340 - INFO -  Fetching Credit Card Emails from 01-Jan-2024 to 01-Jan-2025


...

 2 PDFs downloaded for 2016


2025-08-12 18:23:26,557 - INFO -  Fetching Credit Card Emails from 01-Jan-2015 to 01-Jan-2016



 Processing Year: 2015
Searching from 01-Jan-2015 to 01-Jan-2016...


2025-08-12 18:23:27,775 - INFO -  Successfully connected to email server
2025-08-12 18:23:28,268 - INFO - Searching emails from 01-Jan-2015 to 31-Dec-2015
2025-08-12 18:23:30,026 - INFO -  No emails found in this date range.
2025-08-12 18:23:30,551 - INFO -  Email connection closed


 No emails found for 2015. Stopping search.
 All Years Processing Complete!
Total PDFs downloaded: 195


In [22]:
# Email Body Content Parser - Phase 2 Alternative Approach
import re
from datetime import datetime
import pandas as pd

def extract_amount_from_email_body(email_body, bank):
    """Extract amount from email body text"""
    # Common amount patterns for both banks
    amount_patterns = [
        r'Total Amount Due[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Amount Due[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Amount Payable[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Outstanding Amount[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Current Balance[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        # More flexible patterns
        r'(?:Rs\.?\s*|INR\s*|₹\s*)([0-9,]+\.?[0-9]*)',
        r'([0-9,]+\.?[0-9]*)\s*(?:Rs\.?|INR|₹)'
    ]
    
    for pattern in amount_patterns:
        matches = re.findall(pattern, email_body, re.IGNORECASE)
        if matches:
            for match in matches:
                try:
                    # Clean and convert amount
                    clean_amount = match.replace(',', '').strip()
                    amount = float(clean_amount)
                    # Reasonable amount range for credit card bills
                    if 100 <= amount <= 500000:  # Between Rs.100 to Rs.5,00,000
                        return amount
                except ValueError:
                    continue
    return None

def extract_due_date_from_email_body(email_body):
    """Extract due date from email body"""
    due_date_patterns = [
        r'Due Date[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
        r'Payment Due Date[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
        r'Pay by[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
        r'due on[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})'
    ]
    
    for pattern in due_date_patterns:
        matches = re.findall(pattern, email_body, re.IGNORECASE)
        if matches:
            return matches[0]  # Return first match
    return ""

def standardize_date_components(date_str):
    """Convert date to required format components"""
    if not date_str:
        return "", "", ""
    
    try:
        # Try different date formats
        for fmt in ['%d/%m/%Y', '%d-%m-%Y', '%Y/%m/%d', '%Y-%m-%d']:
            try:
                date_obj = datetime.strptime(date_str, fmt)
                return (
                    date_obj.strftime('%d/%m/%Y'),  # DATE
                    date_obj.strftime('%B'),        # MONTH  
                    str(date_obj.year)              # YEAR
                )
            except ValueError:
                continue
                
        # Manual parsing for mixed formats
        if re.match(r'\d{1,2}[/-]\d{1,2}[/-]\d{4}', date_str):
            parts = re.split('[/-]', date_str)
            day, month, year = parts[0], parts[1], parts[2]
            date_obj = datetime(int(year), int(month), int(day))
            return (
                f"{day.zfill(2)}/{month.zfill(2)}/{year}",
                date_obj.strftime('%B'),
                year
            )
    except:
        pass
    
    return date_str, "", ""

def process_email_body_for_data(msg, sender):
    """Process email message to extract required data"""
    try:
        # Get email body text
        email_body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain":
                    email_body += part.get_payload(decode=True).decode('utf-8', errors='ignore')
        else:
            email_body = msg.get_payload(decode=True).decode('utf-8', errors='ignore')
        
        # Identify bank
        bank = identify_bank(sender)
        if bank not in ['HDFC', 'IDFC']:
            return None
            
        # Extract amount
        amount = extract_amount_from_email_body(email_body, bank)
        if amount is None:
            return None
            
        # Get email date (received date as statement date)
        email_date = msg.get('Date', '')
        if email_date:
            try:
                # Parse email date
                email_datetime = email.utils.parsedate_to_datetime(email_date)
                statement_date = email_datetime.strftime('%d/%m/%Y')
            except:
                statement_date = datetime.now().strftime('%d/%m/%Y')
        else:
            statement_date = datetime.now().strftime('%d/%m/%Y')
            
        # Extract due date from email body
        due_date = extract_due_date_from_email_body(email_body)
        
        # Standardize statement date components
        date_formatted, month_name, year = standardize_date_components(statement_date)
        
        # Create record
        record = {
            'DATE': date_formatted,
            'MONTH': month_name,
            'YEAR': year, 
            'BANK': bank,
            'AMOUNT': amount,
            'DUE_DATE': due_date,
            'EMAIL_SUBJECT': decode_header_value(msg.get("Subject", "")),
            'PROCESSED_TIME': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        return record
        
    except Exception as e:
        logger.error(f"Error processing email body: {e}")
        return None

print(" Email body parsing functions defined successfully!")


 Email body parsing functions defined successfully!


In [23]:
def create_excel_from_email_data(records, output_path):
    """Create Excel file from email body extracted data"""
    if not records:
        print(" No data to export to Excel")
        return False
        
    try:
        # Create DataFrame
        df = pd.DataFrame(records)
        
        # Ensure output directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Create Excel with multiple sheets
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            # Main data sheet - your required columns first
            main_columns = ['DATE', 'MONTH', 'YEAR', 'BANK', 'AMOUNT', 'DUE_DATE']
            df_main = df[main_columns + ['EMAIL_SUBJECT', 'PROCESSED_TIME']]
            df_main.to_excel(writer, sheet_name='Credit Card Data', index=False)
            
            # Summary by bank and year
            summary = df.groupby(['BANK', 'YEAR']).agg({
                'AMOUNT': ['sum', 'count', 'mean']
            }).round(2)
            summary.columns = ['Total_Amount', 'Count', 'Average_Amount']
            summary.to_excel(writer, sheet_name='Bank Year Summary')
            
            # Monthly summary
            monthly = df.groupby(['YEAR', 'MONTH', 'BANK'])['AMOUNT'].sum().reset_index()
            monthly.to_excel(writer, sheet_name='Monthly Summary', index=False)
        
        # Display results
        total_amount = df['AMOUNT'].sum()
        print(f"EXCEL FILE CREATED SUCCESSFULLY!")
        print(f"Location: {output_path}")
        print(f" Total Records: {len(df)}")
        print(f"Banks: {', '.join(df['BANK'].unique())}")
        print(f" Total Amount: ₹{total_amount:,.2f}")
        print(f" Date Range: {df['DATE'].min()} to {df['DATE'].max()}")
        
        return True
        
    except Exception as e:
        print(f" Error creating Excel file: {e}")
        return False

print(" Excel output function defined successfully!")


 Excel output function defined successfully!


In [27]:
# IMPROVED Email Body Content Parser - Phase 2 Alternative Approach
import re
from datetime import datetime
import pandas as pd
from html import unescape

def clean_html_content(html_text):
    """Clean HTML content and extract readable text"""
    if not html_text:
        return ""
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', html_text)
    
    # Decode HTML entities
    text = unescape(text)
    
    # Clean up extra spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    
    # Remove CSS and JavaScript content
    text = re.sub(r'\.[\w\-]+\s*\{[^}]*\}', ' ', text)  # CSS
    text = re.sub(r'function\s*\([^}]*\}', ' ', text)   # JavaScript
    
    return text.strip()

def extract_amount_from_email_body(email_body, bank):
    """Extract amount from email body text - IMPROVED"""
    # Clean HTML first
    clean_body = clean_html_content(email_body)
    
    # IDFC specific patterns (more comprehensive)
    idfc_patterns = [
        r'Total Amount Due[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Amount Due[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Amount Payable[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Outstanding[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Current Balance[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        # Try reverse patterns
        r'(?:Rs\.?\s*|INR\s*|₹\s*)([0-9,]+\.?[0-9]*)',
        # Look for numbers followed by currency in various contexts
        r'([0-9,]+\.?[0-9]*)\s*(?:Rs\.?|INR|₹)',
        # More flexible patterns
        r'\b([0-9,]+\.[0-9]{2})\b',  # Any number with 2 decimal places
        r'\b([1-9][0-9,]{2,})\b'     # Numbers with 3+ digits
    ]
    
    # HDFC specific patterns 
    hdfc_patterns = [
        r'Total Amount Due[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Amount Due[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Outstanding Balance[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'Current Outstanding[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
        r'(?:Rs\.?\s*|INR\s*|₹\s*)([0-9,]+\.?[0-9]*)',
        r'([0-9,]+\.?[0-9]*)\s*(?:Rs\.?|INR|₹)',
        r'\b([0-9,]+\.[0-9]{2})\b',
        r'\b([1-9][0-9,]{2,})\b'
    ]
    
    # Choose patterns based on bank
    patterns = idfc_patterns if bank == 'IDFC' else hdfc_patterns
    
    # Try patterns on both original and cleaned content
    for content in [clean_body, email_body]:
        for pattern in patterns:
            matches = re.findall(pattern, content, re.IGNORECASE)
            if matches:
                for match in matches:
                    try:
                        # Clean and convert amount
                        clean_amount = match.replace(',', '').strip()
                        amount = float(clean_amount)
                        # Reasonable amount range (₹50 to ₹10,00,000)
                        if 50 <= amount <= 1000000:
                            print(f"    Found amount: ₹{amount} using pattern: {pattern[:50]}...")
                            return amount
                    except ValueError:
                        continue
    
    print(f"    No valid amount found for {bank}")
    return None

def extract_due_date_from_email_body(email_body):
    """Extract due date from email body - IMPROVED"""
    clean_body = clean_html_content(email_body)
    
    # Due date patterns - more comprehensive
    due_date_patterns = [
        r'Due Date[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
        r'Payment Due Date[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
        r'Pay by[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
        r'due on[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
        r'Payment date[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
        r'Last date[:\s]*([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',
        # Different date formats
        r'([0-9]{1,2}[/-][0-9]{1,2}[/-][0-9]{4})',  # Any date pattern
        # Month name formats
        r'([0-9]{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+[0-9]{4})',
        r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+[0-9]{1,2},?\s+[0-9]{4})'
    ]
    
    # Try patterns on both cleaned and original content
    for content in [clean_body, email_body]:
        for pattern in due_date_patterns:
            matches = re.findall(pattern, content, re.IGNORECASE)
            if matches:
                # Return first reasonable match
                for match in matches:
                    if len(match) >= 8:  # Minimum reasonable date length
                        print(f"    Found due date: {match}")
                        return match
    
    print("    No due date found")
    return ""

def process_email_body_for_data(msg, sender):
    """Process email message to extract required data - IMPROVED"""
    try:
        # Get email body text (both HTML and plain text)
        email_body = ""
        html_body = ""
        plain_body = ""
        
        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                
                if content_type == "text/plain":
                    try:
                        plain_content = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                        plain_body += plain_content
                    except:
                        pass
                        
                elif content_type == "text/html":
                    try:
                        html_content = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                        html_body += html_content
                    except:
                        pass
        else:
            try:
                content = msg.get_payload(decode=True).decode('utf-8', errors='ignore')
                if msg.get_content_type() == "text/html":
                    html_body = content
                else:
                    plain_body = content
            except:
                pass
        
        # Use HTML body if available, otherwise plain body
        email_body = html_body if html_body else plain_body
        
        if not email_body:
            print("    No email body content found")
            return None
        
        # Identify bank
        bank = identify_bank(sender)
        if bank not in ['HDFC', 'IDFC']:
            print(f"    Unknown bank: {bank}")
            return None
        
        print(f"    Processing {bank} email")
        
        # Extract amount
        amount = extract_amount_from_email_body(email_body, bank)
        if amount is None:
            print(f"    No amount found in {bank} email")
            return None
        
        # Get email date (received date as statement date)
        email_date = msg.get('Date', '')
        if email_date:
            try:
                # Parse email date
                import email.utils
                email_datetime = email.utils.parsedate_to_datetime(email_date)
                statement_date = email_datetime.strftime('%d/%m/%Y')
            except:
                statement_date = datetime.now().strftime('%d/%m/%Y')
        else:
            statement_date = datetime.now().strftime('%d/%m/%Y')
        
        # Extract due date from email body
        due_date = extract_due_date_from_email_body(email_body)
        
        # Standardize statement date components
        date_formatted, month_name, year = standardize_date_components(statement_date)
        
        # Create record
        record = {
            'DATE': date_formatted,
            'MONTH': month_name,
            'YEAR': year,
            'BANK': bank,
            'AMOUNT': amount,
            'DUE_DATE': due_date,
            'EMAIL_SUBJECT': decode_header_value(msg.get("Subject", "")),
            'PROCESSED_TIME': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        print(f"    Successfully extracted: {bank} ₹{amount} {month_name} {year}")
        return record
        
    except Exception as e:
        print(f"    Error processing email body: {e}")
        return None

print(" IMPROVED Email body parsing functions defined successfully!")


 IMPROVED Email body parsing functions defined successfully!


In [28]:
# Run the email body extraction
extracted_records = run_email_body_extraction(CONFIG)

print(f"\n EXTRACTION COMPLETE!")
print(f" Records extracted: {len(extracted_records)}")
if extracted_records:
    sample_record = extracted_records[0]
    print(f" Sample record: {sample_record}")


 STARTING EMAIL BODY DATA EXTRACTION
 Processing email content directly (No PDF downloads)


2025-08-13 00:38:22,480 - INFO -  Successfully connected to email server



 Processing Year: 2025
   Searching: 01-Jan-2025 to 01-Jan-2026


2025-08-13 00:38:23,037 - INFO - Searching emails from 01-Jan-2025 to 31-Dec-2025
2025-08-13 00:38:23,714 - INFO - Found 5 emails with criteria: (FROM "Emailstatements.cards@hdfcbank.net" SINCE "01-Jan-2025" BEFORE "31-Dec-2025")
2025-08-13 00:38:24,663 - INFO - Found 5 emails with criteria: (SUBJECT "statement" SINCE "01-Jan-2025" BEFORE "31-Dec-2025") (FROM "hdfc")
2025-08-13 00:38:24,957 - INFO - Found 17 emails with criteria: (SUBJECT "statement" SINCE "01-Jan-2025" BEFORE "31-Dec-2025") (FROM "idfc")


 Found 22 emails
    Processing IDFC email
    Found amount: ₹721.45 using pattern: Total Amount Due[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0...
    Found due date: 15-11-2022
    Successfully extracted: IDFC ₹721.45 July 2025
    Processing IDFC email
    Found amount: ₹2024.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: IDFC ₹2024.0 January 2025
    Processing IDFC email
    Found amount: ₹2025.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: IDFC ₹2025.0 March 2025
    Processing HDFC email
    Found amount: ₹100.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: HDFC ₹100.0 April 2025
    Processing HDFC email
    Found amount: ₹10198.67 using pattern: \b([0-9,]+\.[0-9]{2})\b...
    Found due date: 04-04-2025
    Successfully extracted: HDFC ₹10198.67 March 2025
    Processing IDFC email
    Found amount: ₹399.0 using pattern: Total Amount Due[:\s]*(?:Rs\.?\s*|I

...

 Found 10 emails
    Processing HDFC email
    Found amount: ₹100.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: HDFC ₹100.0 October 2017
    Processing HDFC email
    Found amount: ₹100.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: HDFC ₹100.0 September 2017
    Processing HDFC email
    Found amount: ₹100.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: HDFC ₹100.0 August 2017
    Processing HDFC email
    Found amount: ₹100.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: HDFC ₹100.0 June 2017
    Processing HDFC email
    Found amount: ₹100.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: HDFC ₹100.0 July 2017
    Processing HDFC email
    Found amount: ₹1999.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: HDFC ₹1999.0 Feb

2025-08-13 00:41:32,433 - INFO - Searching emails from 01-Jan-2016 to 31-Dec-2016
2025-08-13 00:41:33,935 - INFO - Found 2 emails with criteria: (SUBJECT "statement" SINCE "01-Jan-2016" BEFORE "31-Dec-2016") (FROM "hdfc")


 Found 2 emails
    Processing HDFC email
    Found amount: ₹1999.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: HDFC ₹1999.0 November 2016
    Processing HDFC email
    Found amount: ₹1999.0 using pattern: \b([1-9][0-9,]{2,})\b...
    No due date found
    Successfully extracted: HDFC ₹1999.0 December 2016
 Extracted 2 records from 2016

 Processing Year: 2015
   Searching: 01-Jan-2015 to 01-Jan-2016


2025-08-13 00:41:35,928 - INFO - Searching emails from 01-Jan-2015 to 31-Dec-2015


 No emails for 2015. Stopping.
EXCEL FILE CREATED SUCCESSFULLY!
Location: ../data/raw_emails\output\credit_card_email_data.xlsx
 Total Records: 231
Banks: IDFC, HDFC
 Total Amount: ₹977,219.04
 Date Range: 02/01/2024 to 24/12/2022

 EMAIL BODY EXTRACTION COMPLETED!
 Final Results: 231 records extracted

 EXTRACTION COMPLETE!
 Records extracted: 231
 Sample record: {'DATE': '23/07/2025', 'MONTH': 'July', 'YEAR': '2025', 'BANK': 'IDFC', 'AMOUNT': 721.45, 'DUE_DATE': '15-11-2022', 'EMAIL_SUBJECT': 'Your FIRST Millennia Credit Card Statement', 'PROCESSED_TIME': '2025-08-13 00:38:26'}


In [26]:
def debug_single_email(config, max_emails=3):
    """Debug email body extraction with detailed logging"""
    print(" DEBUGGING EMAIL BODY EXTRACTION")
    print("=" * 50)
    
    mail = connect_to_email(config)
    if not mail:
        return
    
    try:
        # Get recent emails for testing
        start_date = datetime(2025, 1, 1)
        end_date = datetime(2025, 12, 31)
        email_ids = search_credit_card_emails_between(mail, config, start_date, end_date)
        
        if not email_ids:
            print(" No emails found for debugging")
            return
            
        print(f" Testing with {min(max_emails, len(email_ids))} recent emails")
        
        for i, email_id in enumerate(email_ids[:max_emails]):
            print(f"\n DEBUG EMAIL #{i+1} (ID: {email_id.decode()})")
            print("-" * 30)
            
            try:
                # Fetch email
                typ, msg_data = mail.fetch(email_id, '(RFC822)')
                
                for response_part in msg_data:
                    if isinstance(response_part, tuple):
                        msg = email.message_from_bytes(response_part[1])
                        
                        # Debug email headers
                        sender = decode_header_value(msg.get("From", ""))
                        subject = decode_header_value(msg.get("Subject", ""))
                        date = msg.get("Date", "")
                        
                        print(f" From: {sender}")
                        print(f" Subject: {subject}")
                        print(f" Date: {date}")
                        
                        # Debug bank identification
                        bank = identify_bank(sender)
                        print(f" Bank Identified: {bank}")
                        
                        # Extract email body with debug info
                        email_body = ""
                        body_parts = []
                        
                        if msg.is_multipart():
                            print(" Multipart email - extracting text parts...")
                            for part in msg.walk():
                                content_type = part.get_content_type()
                                print(f"   Part type: {content_type}")
                                
                                if content_type == "text/plain":
                                    try:
                                        part_content = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                                        email_body += part_content
                                        body_parts.append(f"Plain text: {len(part_content)} chars")
                                    except Exception as e:
                                        print(f"   Error decoding plain text: {e}")
                                        
                                elif content_type == "text/html":
                                    try:
                                        html_content = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                                        # Simple HTML to text conversion
                                        import re
                                        text_content = re.sub('<[^<]+?>', ' ', html_content)
                                        email_body += text_content
                                        body_parts.append(f"HTML text: {len(text_content)} chars")
                                    except Exception as e:
                                        print(f"   Error decoding HTML: {e}")
                        else:
                            print(" Single part email...")
                            try:
                                email_body = msg.get_payload(decode=True).decode('utf-8', errors='ignore')
                                body_parts.append(f"Single part: {len(email_body)} chars")
                            except Exception as e:
                                print(f"   Error decoding single part: {e}")
                        
                        print(f" Body parts found: {body_parts}")
                        print(f" Total body length: {len(email_body)} characters")
                        
                        if email_body:
                            # Show first 500 characters of body for debugging
                            print(f" Body preview (first 500 chars):")
                            print("   " + email_body[:500].replace('\n', ' ').replace('\r', ' '))
                            print()
                            
                            # Test amount extraction with debug
                            print(" TESTING AMOUNT EXTRACTION:")
                            amount = extract_amount_from_email_body(email_body, bank)
                            print(f"   Amount found: {amount}")
                            
                            # Show what patterns we're looking for
                            print("   Testing patterns:")
                            amount_patterns = [
                                r'Total Amount Due[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
                                r'Amount Due[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
                                r'Amount Payable[:\s]*(?:Rs\.?\s*|INR\s*|₹\s*)?([0-9,]+\.?[0-9]*)',
                                r'(?:Rs\.?\s*|INR\s*|₹\s*)([0-9,]+\.?[0-9]*)'
                            ]
                            
                            for pattern in amount_patterns:
                                matches = re.findall(pattern, email_body, re.IGNORECASE)
                                if matches:
                                    print(f"   Pattern matched: {pattern}")
                                    print(f"      Matches: {matches}")
                                    break
                            else:
                                print("  No amount patterns matched")
                            
                            # Test due date extraction
                            print(" TESTING DUE DATE EXTRACTION:")
                            due_date = extract_due_date_from_email_body(email_body)
                            print(f"   Due date found: {due_date}")
                            
                        else:
                            print(" No email body content extracted!")
                            
            except Exception as e:
                print(f" Error processing email {i+1}: {e}")
                import traceback
                traceback.print_exc()
                
    finally:
        try:
            mail.close()
            mail.logout()
        except:
            pass

# Run debugging
debug_single_email(CONFIG, max_emails=2)


 DEBUGGING EMAIL BODY EXTRACTION


2025-08-13 00:32:27,371 - INFO -  Successfully connected to email server
2025-08-13 00:32:27,887 - INFO - Searching emails from 01-Jan-2025 to 31-Dec-2025
2025-08-13 00:32:28,469 - INFO - Found 5 emails with criteria: (FROM "Emailstatements.cards@hdfcbank.net" SINCE "01-Jan-2025" BEFORE "31-Dec-2025")
2025-08-13 00:32:29,389 - INFO - Found 5 emails with criteria: (SUBJECT "statement" SINCE "01-Jan-2025" BEFORE "31-Dec-2025") (FROM "hdfc")
2025-08-13 00:32:29,678 - INFO - Found 17 emails with criteria: (SUBJECT "statement" SINCE "01-Jan-2025" BEFORE "31-Dec-2025") (FROM "idfc")


 Testing with 2 recent emails

 DEBUG EMAIL #1 (ID: 84442)
------------------------------
 From: IDFC FIRST Bank <statement@idfcfirstbank.com>
 Subject: Your FIRST Millennia Credit Card Statement
 Date: Wed, 23 Jul 2025 22:40:42 +0530
 Bank Identified: IDFC
 Multipart email - extracting text parts...
   Part type: multipart/mixed
   Part type: multipart/related
   Part type: multipart/alternative
   Part type: text/html
   Part type: application/octet-stream
 Body parts found: ['HTML text: 3101 chars']
 Total body length: 3101 characters
 Body preview (first 500 chars):
            IDFC FIRST Bank           outlookfix{ width:0 !important;}  table {border-spacing: 0 !important; border-collapse: collapse !important;}  img{max-width:100%; height:auto; display:block; border:0px;} ..clear{display:none !important;}                                                        Dear Cardmember,   We hope transacting with your FIRST Millennia Credit Card ending with XXXX1344 has been a smooth-sailing 